In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

ohe = OneHotEncoder()
X_train = train.iloc[:, 1:]
y_train = train.iloc[:, 0]
X_test = test.iloc[:, 1:]

X_total = pd.concat([X_train, X_test])

X_total_ohe = ohe.fit_transform(X_total)

n_train = X_train.shape[0]

X_train_ohe = X_total_ohe[:n_train,]

X_test_ohe = X_total_ohe[n_train:,]



In [4]:
def make_submission_file(y_test_pred, filename=None):
    submit_data = pd.DataFrame(columns=['ID', 'ACTION'])
    submit_data['ID'] = xrange(1, len(y_test_pred)+1)
    submit_data['ACTION'] = y_test_pred
    if not filename:
        filename = 'submit.csv'
    submit_data.to_csv(filename, index=False)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC

clf_rf = RandomForestClassifier()
clf_logr = LogisticRegression()
clf_svm = SVC(probability=True)




In [6]:
from mlxtend.classifier import StackingClassifier

lr = LinearRegression()
clf_stacking = StackingClassifier(classifiers=[clf_rf, clf_logr, clf_svm], 
                          meta_classifier=lr)

ImportError: No module named mlxtend.classifier

In [7]:
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

def scoring(estimator, X, y):
    #y_pred = estimator.predict_proba(X)
    #return roc_auc_score(y, y_pred[:, 1])
    y_pred = estimator.predict(X)
    return roc_auc_score(y, y_pred)

scores = model_selection.cross_val_score(clf_stacking, X_train_ohe, y_train, 
                                              cv=3, scoring=scoring)
scores

NameError: name 'clf_stacking' is not defined

In [5]:
from sklearn.base import BaseEstimator

class StackingClassifier1(BaseEstimator):
    
    def __init__(self, clfs_stage1, clf_stage2):
        self.clfs_stage1 = clfs_stage1
        self.clf_stage2 = clf_stage2
        
        
    def fit(self, X, y):
        X_tr, X_cv, y_tr, y_cv = model_selection.train_test_split(X, y, test_size=0.33)
        
        pred_stage1 = []
        for clf in self.clfs_stage1:
            clf.fit(X_tr, y_tr)
            pred_stage1.append(clf.predict_proba(X_cv)[:, 1].reshape(-1, 1))
        print([i.shape for i in pred_stage1])
        pred_stage1_stacked = np.hstack(pred_stage1)
        print(pred_stage1_stacked.shape)
        print
        
        self.clf_stage2.fit(pred_stage1_stacked, y_cv)
    
    
    def predict_proba(self, X):
        pred_stage1 = []
        for clf in self.clfs_stage1:
            pred_stage1.append(clf.predict_proba(X)[:, 1].reshape(-1, 1))
            
        pred_stage1_stacked = np.hstack(pred_stage1)
        
        clf_stage2 = self.clf_stage2
        if hasattr(clf_stage2, 'predict_proba'):
            return self.clf_stage2.predict_proba(pred_stage1_stacked)[:, 1]
        else:
            return self.clf_stage2.predict(pred_stage1_stacked)
        
    def predict(self, X):
        return self.predict_proba(X)
    

In [24]:

np.logspace(-1, 1, 10)

array([  0.1       ,   0.16681005,   0.27825594,   0.46415888,
         0.77426368,   1.29154967,   2.15443469,   3.59381366,
         5.9948425 ,  10.        ])

In [27]:
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import RidgeCV

logr = LogisticRegression(C=3)
lr = RidgeCV(alphas=np.logspace(-1, 1, 10), cv=10)
clf_etc = ExtraTreesClassifier()
clf_gbc = GradientBoostingClassifier()
clf_stacking = StackingClassifier1([clf_rf, clf_logr, clf_etc], lr)

In [28]:
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

def scoring_roc_auc(estimator, X, y):
    #y_pred = estimator.predict_proba(X)
    #return roc_auc_score(y, y_pred[:, 1])
    y_pred = estimator.predict(X)
    return roc_auc_score(y, y_pred)

scores = model_selection.cross_val_score(clf_stacking, X_train_ohe, y_train, 
                                              cv=3, scoring=scoring_roc_auc, n_jobs=4)
scores

[(7210, 1), (7210, 1), (7210, 1)]
(7210, 3)

[(7210, 1), (7210, 1), (7210, 1)]
(7210, 3)

[(7210, 1), (7210, 1), (7210, 1)]
(7210, 3)



array([ 0.85183368,  0.83133105,  0.84840628])

In [11]:
model_selection.cross_val_score?

In [68]:
a1 = np.array(range(5))

In [78]:
np.hstack([a1.reshape(-1, 1), a1.reshape(-1, 1)])

array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4]])

In [119]:
clf_stacking.fit(X_train_ohe, y_train)
y_test_pred = clf_stacking.predict(X_test_ohe)
make_submission_file(y_test_pred)


[(10814, 1), (10814, 1)]
(10814, 2)



In [121]:
!ls -l submit.csv


-rw-rw-r-- 1 amir amir 1219868 May 24 18:40 submit.csv
